## Imports

In [1]:
#importing necessary libraries
from api.dani_mapbox_api_walking import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
from pyproj import Geod

## Importing Coverage Function (allows me to pick any branch I want)

In [2]:
branchInfo = gpd.read_file('../../can-summer-2023/data/clean/branches.csv')
censusTracts = pd.read_csv('../data/clean/population_demographics.csv')
censusDemographics = pd.read_csv('../data/final_data/census_demos.csv')

In [3]:
#removing unecessary columns from branchInfo
branchInfo = branchInfo.drop(columns = ['HOURS OF OPERATION', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE', 'WEBSITE','PERMANENTLY_CLOSED'])

#removing unecessary columns from censusTracts
censusTracts = censusTracts[['geoid','geometry','qualifying name']]

In [4]:
branchInfo

,BRANCH,LOCATION,geometry
0,Albany Park,"(41.97557881655979, -87.71361314512697)",None
1,Altgeld,"(41.65719847558056, -87.59883829075173)",None
2,Archer Heights,"(41.80110836194246, -87.72648385568911)",None
3,Austin,"(41.88917312206642, -87.7658203582574)",None
4,Austin-Irving,"(41.95312267684315, -87.77928489355646)",None
...,...,...,...
77,West Pullman,"(41.67790647382097, -87.6431683153105)",None
78,West Town,"(41.89594730478282, -87.66829305460979)",None
79,"Whitney M. Young, Jr.","(41.7510184591363, -87.61495065361933)",None
80,Woodson Regional Library,"(41.720694885749005, -87.64304817213312)",None


In [5]:
#converting censusTracts to a geodataframe (we could not load it as one since it already had a geometry column so we are simply
#converting the geometry column to a proper gpd geometry column).
censusTracts = gpd.GeoDataFrame(
    censusTracts.loc[:, [c for c in censusTracts.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(censusTracts["geometry"]),
    crs="epsg:4326",
    )

In [6]:
branchInfo[['LATITUDE', 'LONGITUDE']] = [ast.literal_eval(x)[:2] for x in branchInfo['LOCATION']]
branchInfo.loc[:, 'LATITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LATITUDE'])
branchInfo.loc[:, 'LONGITUDE'] = pd.to_numeric(branchInfo.loc[:, 'LONGITUDE'])

In [7]:
#helper method to ensure that two polygons overlap before getting the intersection. Gets the intersection then calculates
#the percent of overlapping area for a census tract and saves it to a dictionary with the census tract geo-id as the key and
#the percent as the value.
def check(polygon1, df, geoname, dictionary,key):
   
    for i in range(len(df)):
        if polygon1.intersects(df.loc[i,geoname]): 
            overlapPolygon = (polygon1.intersection(df.loc[i,geoname]))
            poly_area, poly_perimeter = geod.geometry_area_perimeter(overlapPolygon)
            overlapArea = poly_area*-1
            propOverlap = overlapArea / df.loc[i,'Area']
            #print(propOverlap)
            dictionary[df.loc[i, key]] = propOverlap
    return dictionary
  

In [8]:
#sets the crs of gpd types
def geoSetup(gdf):
    
    gdf = gdf.set_crs('EPSG:4326')
    

In [9]:
#cdf = gdf of geometry area which you want to get the coverage score
#idf = gdf of points for which you want to get an isochrone and dictionary of percent of overlap in cdf areas
#scoreString = a string that is the name of the coverage score column you choose
#dictString = a string that is the name of the dictionary column you choose
#cgs = a string that is the name of the geometry column in the cdf
#igs = a string that is the name of the geometry column in the idf
#lat = a string that is the name of the latitude column in the idf
#lon = a string that is the name of the longitude column in the idf
#key = a string that is the name of the column in the cdf that you want to represent the key of the dictionary
#travelTime = an integer of the amount of time traveled you wish to calculate the isochrone

def appendADS(cdf,idf,scoreString,dictString,cgs,igs,lat,lon,key,travelTime):
    
    api = MapboxAPI()
    cdf[scoreString] = 0.0
    idf[dictString] = ''
    censusAreas = []
    global geod 
    geod = Geod(ellps='WGS84')
   
    geoSetup(cdf)
    geoSetup(idf)
   
    for index, row in cdf.iterrows():
        poly_area, poly_perimeter = geod.geometry_area_perimeter(row[cgs])
        poly_area = poly_area*-1
        censusAreas.append(poly_area)
    
    cdf['Area'] = censusAreas



    for i in range(len(idf)):
        dictionary = {}
        api_response =  api.get_data((idf.loc[i,lat],idf.loc[i,lon]), travelTime)
        idf.loc[i,igs] = shape(api_response)
        dictionary = check(idf.loc[i,igs],cdf,cgs, dictionary,key)
        branchInfo.loc[i,dictString]= [dictionary]
    
        for i in range(len(cdf)):
   
            if dictionary.get(cdf.loc[i, key]) != None:
                score = cdf.loc[i, scoreString]
                cdf.loc[i, scoreString] = score + dictionary.get(cdf.loc[i, key])
  
    

In [10]:
appendADS(censusTracts,branchInfo, 'score', 'list of dict','geometry','geometry','LATITUDE','LONGITUDE','geoid',20)


### 6 Individual Functions

In [11]:
#Internet
def internet_by_branch(DemographicsVariables, branchWithCensus):
    ''' We are creating a function that allows me to retrieve only the internet census data, and then compare this with Altgeld'''
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['Total']).sum()
    Branch_pop_withInternet = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['Total']).sum() 
    result = Branch_pop_withInternet/pop_in_branch
    return result   

#Income
def income_by_branch(DemographicsVariables, branchWithCensus):
    if DemographicsVariables == 'Percent Population in Labor Force 16+: Employed' or DemographicsVariables == 'Percent Population in Labor Force 16+: Unemployed':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['civilian population in labor force 16 years and over']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['civilian population in labor force 16 years and over']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Households: Less Than $10,000' or DemographicsVariables == 'Percent Households: $10,000 to $14,999' or DemographicsVariables == 'Percent Households: $15,000 to $19,999' or DemographicsVariables == 'Percent Households: $20,000 to $24,999' or DemographicsVariables == 'Percent Households: $25,000 to $29,999' or DemographicsVariables == 'Percent Households: $30,000 to $34,999' or DemographicsVariables == 'Percent Households: $35,000 to $39,999' or DemographicsVariables == 'Percent Households: $40,000 to $44,999' or DemographicsVariables == 'Percent Households: $45,000 to $49,999' or DemographicsVariables == 'Percent Households: $50,000 to $59,999' or DemographicsVariables == 'Percent Households: $60,000 to $74,999' or DemographicsVariables == 'Percent Households: $75,000 to $99,999' or DemographicsVariables == 'Percent Households: $100,000 to $124,999' or DemographicsVariables == 'Percent Households: $125,000 to $149,999' or DemographicsVariables == 'Percent Households: $150,000 to $199,999' or DemographicsVariables == 'Percent Households: $200,000 or more':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total households reporting income']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total households reporting income']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Ages 18-64: Living in Poverty' or DemographicsVariables == 'Percent Ages 18-64: At or Above Poverty Level':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['population age 18 to 64 for whom poverty status  is determined']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['population age 18 to 64 for whom poverty status  is determined']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Households: With Public Assistance Income' or DemographicsVariables == 'Percent Households: No Public Assistance Income':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total households reporting public assistance income status']).sum()
        Branch_pop_withIncome = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total households reporting public assistance income status']).sum() 
        result = Branch_pop_withIncome/pop_in_branch
        return result

# #Commute
def commute_by_branch(DemographicsVariables, branchWithCensus):
    if DemographicsVariables == 'Percent Owner and Renter Occupied Housing: No Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 1 Vehicle' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 2 Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 3 Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 4 Vehicles' or DemographicsVariables == 'Percent Owner and Renter Occupied Housing: 5 or More Vehicles':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['occupied housing units']).sum()
        Branch_pop_withCommute = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['occupied housing units']).sum() 
        result = Branch_pop_withCommute/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Renter-Occupied Housing: No Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 1 Vehicle' or DemographicsVariables == 'Percent Renter-Occupied Housing: 2 Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 3 Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 4 Vehicles' or DemographicsVariables == 'Percent Renter-Occupied Housing: 5 or More Vehicles':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['renter-occupied housing units']).sum()
        Branch_pop_withCommute = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['renter-occupied housing units']).sum() 
        result = Branch_pop_withCommute/pop_in_branch
        return result
    elif DemographicsVariables == 'Percent Workers 16+: Car, Truck, or Van ' or DemographicsVariables == 'Percent Workers 16+: Drove Alone' or DemographicsVariables == 'Percent Workers 16+: Carpooled' or DemographicsVariables == 'Percent Workers 16+: Public Transportation (Includes Taxi)' or DemographicsVariables == 'Percent Workers 16+: Motorcycle' or DemographicsVariables == 'Percent Workers 16+: Bicycle' or DemographicsVariables == 'Percent Workers 16+: Walked' or DemographicsVariables == 'Percent Workers 16+: Other Means':
        print(DemographicsVariables)
        pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['workers 16 years and over']).sum()
        Branch_pop_withCommute = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['workers 16 years and over']).sum() 
        result = Branch_pop_withCommute/pop_in_branch
        return result

#Race
def race_by_branch(DemographicsVariables, branchWithCensus):
    """ This function takes the population of each variables and devide it by the population to give us the perecntage of how many people have a specific Race in the overlapping area of the branch"""
    print(DemographicsVariables)
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total population']).sum()
    Branch_pop_withRace = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total population']).sum()
    result = Branch_pop_withRace/pop_in_branch
    return result

#Age
def age_by_branch(DemographicsVariables, branchWithCensus):
    """ This function takes the population of each variables and devide it by the population to give us the perecntage of how many people have a specific age in the overlapping area of the branch"""
    print(DemographicsVariables)
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['total population']).sum()
    Branch_pop_withAge = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['total population']).sum()
    result = Branch_pop_withAge/pop_in_branch
    return result

#Education
def education_by_branch(DemographicsVariables, branchWithCensus):
    """ This function takes the population of each variables and devide it by the population to give us the perecntage of how many people have a specific degree in the overlapping area of the branch"""
    print(DemographicsVariables)
    pop_in_branch = (branchWithCensus['Percent Overlap']  * branchWithCensus['population 25 years and over']).sum()
    Branch_pop_withEducation = (branchWithCensus['Percent Overlap'] * branchWithCensus[DemographicsVariables] * branchWithCensus['population 25 years and over']).sum()
    result = Branch_pop_withEducation/pop_in_branch
    return result


### Branch Demographics combining 6 functions

In [12]:
def branch_demographics(branchWithCensus, nameOfBranch):
    branch_snapshot_df = pd.DataFrame({'Branch': [nameOfBranch]})
    #branch_snapshot_df = pd.DataFrame({'Branch': [merge_function.loc[0,'BRANCH']]})
    list1 = ['Percent of People with Internet Access',
             'Percent of People without Internet Access',
             'Percent of People with Computers', 
             'Percent of People without Computers', 
             'Percent of People with Computers and Internet']
    for i in list1:
        branch_snapshot_df[i] = internet_by_branch(i, branchWithCensus)
    
    list2 = ['Percent Population in Labor Force 16+: Employed',
             'Percent Population in Labor Force 16+: Unemployed',
             'Percent Households: Less Than $10,000',
             'Percent Households: $10,000 to $14,999',
             'Percent Households: $15,000 to $19,999',
             'Percent Households: $20,000 to $24,999',
             'Percent Households: $25,000 to $29,999',
             'Percent Households: $30,000 to $34,999',
             'Percent Households: $35,000 to $39,999',
             'Percent Households: $40,000 to $44,999',
             'Percent Households: $45,000 to $49,999',
             'Percent Households: $50,000 to $59,999',
             'Percent Households: $60,000 to $74,999',
             'Percent Households: $75,000 to $99,999',
             'Percent Households: $100,000 to $124,999',
             'Percent Households: $125,000 to $149,999',
             'Percent Households: $150,000 to $199,999',
             'Percent Households: $200,000 or more',
             'Percent Ages 18-64: Living in Poverty',
             'Percent Ages 18-64: At or Above Poverty Level',
             'Percent Households: With Public Assistance Income',
             'Percent Households: No Public Assistance Income']
    for i in list2:
        branch_snapshot_df[i] = income_by_branch(i, branchWithCensus)
    
    list3 = ['Percent Owner and Renter Occupied Housing: No Vehicles',
             'Percent Owner and Renter Occupied Housing: 1 Vehicle',
             'Percent Owner and Renter Occupied Housing: 2 Vehicles',
             'Percent Owner and Renter Occupied Housing: 3 Vehicles',
             'Percent Owner and Renter Occupied Housing: 4 Vehicles',
             'Percent Owner and Renter Occupied Housing: 5 or More Vehicles',
             'Percent Renter-Occupied Housing: No Vehicles',
             'Percent Renter-Occupied Housing: 1 Vehicle',
             'Percent Renter-Occupied Housing: 2 Vehicles',
             'Percent Renter-Occupied Housing: 3 Vehicles',
             'Percent Renter-Occupied Housing: 4 Vehicles',
             'Percent Renter-Occupied Housing: 5 or More Vehicles',
             'Percent Workers 16+: Car, Truck, or Van ',
             'Percent Workers 16+: Drove Alone',
             'Percent Workers 16+: Carpooled',
             'Percent Workers 16+: Public Transportation (Includes Taxi)',
             'Percent Workers 16+: Motorcycle',
             'Percent Workers 16+: Bicycle',
             'Percent Workers 16+: Walked',
             'Percent Workers 16+: Other Means']
    for i in list3:
        branch_snapshot_df[i] = commute_by_branch(i, branchWithCensus)
    
    list4 = ['Percent: White Alone',
             'Percent: Black or African American Alone',
             'Percent: American Indian and Alaska Native Alone',
             'Percent: Asian Alone',
             'Percent: Native Hawaiian and Other Pacific Islander Alone',
             'Percent: Some Other Race Alone',
             'Percent: Two or More Races']
    for i in list4:
        branch_snapshot_df[i] = race_by_branch(i, branchWithCensus)
        
    list5 = ['Percent: under 5 years',
             'Percent: 5 to 9 years',
             'Percent: 10 to 14 years',
             'Percent: 15 to 19 years',
             'Percent: 20 to 24 years',
             'Percent: 25 to 29 years',
             'Percent: 30 to 34 years',
             'Percent: 35 to 39 years',
             'Percent: 50 to 54 years',
             'Percent: 55 to 59 years',
             'Percent: 60 to 64 years',
             'Percent: 65 to 69 years',
             'Percent: 70 to 74 years',
             'Percent: 75 to 79 years',
             'Percent: 80 to 84 years',
             'Percent: 85 years and over']
    for i in list5:
        branch_snapshot_df[i] = age_by_branch(i, branchWithCensus)
    
    list6 = ['Percent: 25+ Less Than High School', 
             'Percent: 25+ High School or More', 
             'Percent: 25+ Some College or More', 
             'Percent: 25+ Bachelor\'s Degree or More', 
             'Percent: 25+ Master\'s Degree or More',
             'Percent: 25+ Professional School Degree or More',
             'Percent: 25+ Doctorate Degree']
    for i in list6: 
        branch_snapshot_df[i] = education_by_branch(i, branchWithCensus)
       
    return branch_snapshot_df

## Final Function 

In [13]:
def allBranchDemographics(branchDataset):
    dataByBranch = pd.DataFrame(columns=['Branch',
                                        'Percent of People with Internet Access',
                                        'Percent of People without Internet Access',
                                        'Percent of People with Computers',
                                        'Percent of People without Computers',
                                        'Percent of People with Computers and Internet',
                                        'Percent Population in Labor Force 16+: Employed',
                                        'Percent Population in Labor Force 16+: Unemployed',
                                        'Percent Households: Less Than $10,000',
                                        'Percent Households: $10,000 to $14,999',
                                        'Percent Households: $15,000 to $19,999',
                                        'Percent Households: $20,000 to $24,999',
                                        'Percent Households: $25,000 to $29,999',
                                        'Percent Households: $30,000 to $34,999',
                                        'Percent Households: $35,000 to $39,999',
                                        'Percent Households: $40,000 to $44,999',
                                        'Percent Households: $45,000 to $49,999',
                                        'Percent Households: $50,000 to $59,999',
                                        'Percent Households: $60,000 to $74,999',
                                        'Percent Households: $75,000 to $99,999',
                                        'Percent Households: $100,000 to $124,999',
                                        'Percent Households: $125,000 to $149,999',
                                        'Percent Households: $150,000 to $199,999',
                                        'Percent Households: $200,000 or more',
                                        'Percent Ages 18-64: Living in Poverty',
                                        'Percent Ages 18-64: At or Above Poverty Level',
                                        'Percent Households: With Public Assistance Income',
                                        'Percent Households: No Public Assistance Income',
                                        'Percent Owner and Renter Occupied Housing: No Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 1 Vehicle',
                                        'Percent Owner and Renter Occupied Housing: 2 Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 3 Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 4 Vehicles',
                                        'Percent Owner and Renter Occupied Housing: 5 or More Vehicles',
                                        'Percent Renter-Occupied Housing: No Vehicles',
                                        'Percent Renter-Occupied Housing: 1 Vehicle',
                                        'Percent Renter-Occupied Housing: 2 Vehicles',
                                        'Percent Renter-Occupied Housing: 3 Vehicles',
                                        'Percent Renter-Occupied Housing: 4 Vehicles',
                                        'Percent Renter-Occupied Housing: 5 or More Vehicles',
                                        'Percent Workers 16+: Car, Truck, or Van ',
                                        'Percent Workers 16+: Drove Alone',
                                        'Percent Workers 16+: Carpooled',
                                        'Percent Workers 16+: Public Transportation (Includes Taxi)',
                                        'Percent Workers 16+: Motorcycle',
                                        'Percent Workers 16+: Bicycle',
                                        'Percent Workers 16+: Walked',
                                        'Percent Workers 16+: Other Means',
                                        'Percent: White Alone',
                                        'Percent: Black or African American Alone',
                                        'Percent: American Indian and Alaska Native Alone',
                                        'Percent: Asian Alone',
                                        'Percent: Native Hawaiian and Other Pacific Islander Alone',
                                        'Percent: Some Other Race Alone',
                                        'Percent: Two or More Races',
                                        'Percent: under 5 years',
                                        'Percent: 5 to 9 years',
                                        'Percent: 10 to 14 years',
                                        'Percent: 15 to 19 years',
                                        'Percent: 20 to 24 years',
                                        'Percent: 25 to 29 years',
                                        'Percent: 30 to 34 years',
                                        'Percent: 35 to 39 years',
                                        'Percent: 50 to 54 years',
                                        'Percent: 55 to 59 years',
                                        'Percent: 60 to 64 years',
                                        'Percent: 65 to 69 years',
                                        'Percent: 70 to 74 years',
                                        'Percent: 75 to 79 years',
                                        'Percent: 80 to 84 years',
                                        'Percent: 85 years and over',
                                        'Percent: 25+ Less Than High School', 
                                        'Percent: 25+ High School or More', 
                                        'Percent: 25+ Some College or More', 
                                        'Percent: 25+ Bachelor\'s Degree or More', 
                                        'Percent: 25+ Master\'s Degree or More',
                                        'Percent: 25+ Professional School Degree or More',
                                        'Percent: 25+ Doctorate Degree'])
    
    for i in range(len(branchDataset)):
        
        #Locating only Altgeld's column 'list of dict'
        libraryDictList = branchDataset.loc[i,'list of dict']
        branchName = branchDataset.loc[i,'BRANCH']
        
        #Removed set of brackets, just a dictionary now
        finalDictionary = libraryDictList[0]

        #Seperating into two columns
        dictToList = pd.DataFrame(finalDictionary.items(), columns= ['geoid','Percent Overlap'])
        libraryAndCensus = censusDemographics.merge(dictToList, on='geoid')

        weightedLibDemographics = branch_demographics(libraryAndCensus, branchName)
        
        dataByBranch = pd.concat([dataByBranch, weightedLibDemographics])
    dataByBranch = dataByBranch.reset_index()
    dataByBranch = dataByBranch.drop(columns = ['index'])
    return dataByBranch

In [14]:
walking20Demographics = allBranchDemographics(branchInfo)
walking20Demographics

Percent Population in Labor Force 16+: Employed
Percent Population in Labor Force 16+: Unemployed
Percent Households: Less Than $10,000
Percent Households: $10,000 to $14,999
Percent Households: $15,000 to $19,999
Percent Households: $20,000 to $24,999
Percent Households: $25,000 to $29,999
Percent Households: $30,000 to $34,999
Percent Households: $35,000 to $39,999
Percent Households: $40,000 to $44,999
Percent Households: $45,000 to $49,999
Percent Households: $50,000 to $59,999
Percent Households: $60,000 to $74,999
Percent Households: $75,000 to $99,999
Percent Households: $100,000 to $124,999
Percent Households: $125,000 to $149,999
Percent Households: $150,000 to $199,999
Percent Households: $200,000 or more
Percent Ages 18-64: Living in Poverty
Percent Ages 18-64: At or Above Poverty Level
Percent Households: With Public Assistance Income
Percent Households: No Public Assistance Income
Percent Owner and Renter Occupied Housing: No Vehicles
Percent Owner and Renter Occupied Hous

,Branch,Percent of People with Internet Access,Percent of People without Internet Access,Percent of People with Computers,Percent of People without Computers,Percent of People with Computers and Internet,Percent Population in Labor Force 16+: Employed,Percent Population in Labor Force 16+: Unemployed,"Percent Households: Less Than $10,000","Percent Households: $10,000 to $14,999",...,Percent: 75 to 79 years,Percent: 80 to 84 years,Percent: 85 years and over,Percent: 25+ Less Than High School,Percent: 25+ High School or More,Percent: 25+ Some College or More,Percent: 25+ Bachelor's Degree or More,Percent: 25+ Master's Degree or More,Percent: 25+ Professional School Degree or More,Percent: 25+ Doctorate Degree
0,Albany Park,0.910526,0.089474,0.952159,0.047841,0.845567,0.939509,0.060491,0.052921,0.027657,...,0.016438,0.010552,0.012915,0.199492,0.800508,0.605922,0.426798,0.153724,0.039378,0.014159
1,Altgeld,0.858834,0.141166,0.919381,0.080619,0.842875,0.735115,0.264885,0.387062,0.076630,...,0.004404,0.005772,0.008001,0.132315,0.867685,0.493944,0.066593,0.032298,0.000290,0.000069
2,Archer Heights,0.809424,0.190576,0.826940,0.173060,0.771128,0.890129,0.109871,0.030794,0.038965,...,0.028900,0.010510,0.019851,0.271524,0.728476,0.349166,0.137559,0.036586,0.004183,0.002146
3,Austin,0.797491,0.202509,0.844575,0.153619,0.722608,0.856880,0.143120,0.142611,0.096485,...,0.028319,0.018925,0.013254,0.136457,0.863543,0.513717,0.204074,0.098090,0.022077,0.009098
4,Austin-Irving,0.871980,0.128020,0.929262,0.070738,0.842911,0.953809,0.046191,0.038843,0.025744,...,0.028754,0.014099,0.019525,0.104904,0.895096,0.597684,0.301560,0.102615,0.025319,0.011787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,West Pullman,0.810429,0.189571,0.874265,0.125735,0.783368,0.801162,0.198838,0.115687,0.065042,...,0.030816,0.030802,0.019957,0.113995,0.886005,0.565888,0.169241,0.067574,0.015149,0.007237
78,West Town,0.889050,0.052455,0.781479,0.038383,0.915111,0.965315,0.034685,0.036508,0.023307,...,0.008220,0.007095,0.004181,0.044911,0.955089,0.882632,0.754532,0.289878,0.095159,0.028578
79,"Whitney M. Young, Jr.",0.752225,0.247775,0.879152,0.120848,0.714651,0.830945,0.169055,0.157410,0.101341,...,0.020387,0.028769,0.025720,0.120488,0.879512,0.642101,0.249856,0.100674,0.027321,0.007505
80,Woodson Regional Library,0.856929,0.143071,0.907988,0.092012,0.822014,0.857066,0.142934,0.098261,0.052483,...,0.027487,0.034408,0.038340,0.113512,0.886488,0.616666,0.255830,0.104982,0.010528,0.007187


In [15]:
walking20Demographics

,Branch,Percent of People with Internet Access,Percent of People without Internet Access,Percent of People with Computers,Percent of People without Computers,Percent of People with Computers and Internet,Percent Population in Labor Force 16+: Employed,Percent Population in Labor Force 16+: Unemployed,"Percent Households: Less Than $10,000","Percent Households: $10,000 to $14,999",...,Percent: 75 to 79 years,Percent: 80 to 84 years,Percent: 85 years and over,Percent: 25+ Less Than High School,Percent: 25+ High School or More,Percent: 25+ Some College or More,Percent: 25+ Bachelor's Degree or More,Percent: 25+ Master's Degree or More,Percent: 25+ Professional School Degree or More,Percent: 25+ Doctorate Degree
0,Albany Park,0.910526,0.089474,0.952159,0.047841,0.845567,0.939509,0.060491,0.052921,0.027657,...,0.016438,0.010552,0.012915,0.199492,0.800508,0.605922,0.426798,0.153724,0.039378,0.014159
1,Altgeld,0.858834,0.141166,0.919381,0.080619,0.842875,0.735115,0.264885,0.387062,0.076630,...,0.004404,0.005772,0.008001,0.132315,0.867685,0.493944,0.066593,0.032298,0.000290,0.000069
2,Archer Heights,0.809424,0.190576,0.826940,0.173060,0.771128,0.890129,0.109871,0.030794,0.038965,...,0.028900,0.010510,0.019851,0.271524,0.728476,0.349166,0.137559,0.036586,0.004183,0.002146
3,Austin,0.797491,0.202509,0.844575,0.153619,0.722608,0.856880,0.143120,0.142611,0.096485,...,0.028319,0.018925,0.013254,0.136457,0.863543,0.513717,0.204074,0.098090,0.022077,0.009098
4,Austin-Irving,0.871980,0.128020,0.929262,0.070738,0.842911,0.953809,0.046191,0.038843,0.025744,...,0.028754,0.014099,0.019525,0.104904,0.895096,0.597684,0.301560,0.102615,0.025319,0.011787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,West Pullman,0.810429,0.189571,0.874265,0.125735,0.783368,0.801162,0.198838,0.115687,0.065042,...,0.030816,0.030802,0.019957,0.113995,0.886005,0.565888,0.169241,0.067574,0.015149,0.007237
78,West Town,0.889050,0.052455,0.781479,0.038383,0.915111,0.965315,0.034685,0.036508,0.023307,...,0.008220,0.007095,0.004181,0.044911,0.955089,0.882632,0.754532,0.289878,0.095159,0.028578
79,"Whitney M. Young, Jr.",0.752225,0.247775,0.879152,0.120848,0.714651,0.830945,0.169055,0.157410,0.101341,...,0.020387,0.028769,0.025720,0.120488,0.879512,0.642101,0.249856,0.100674,0.027321,0.007505
80,Woodson Regional Library,0.856929,0.143071,0.907988,0.092012,0.822014,0.857066,0.142934,0.098261,0.052483,...,0.027487,0.034408,0.038340,0.113512,0.886488,0.616666,0.255830,0.104982,0.010528,0.007187
